In [ ]:
import os
from dotenv import load_dotenv
from app.api_client import get_consumption_load
from datetime import datetime, timedelta
import pandas as pd

# Charge les variables depuis le fichier .env
load_dotenv()

# Récupérer le token
API_TOKEN = os.getenv("API_TOKEN")
PRM = os.getenv("PRM")



start_date = "2024-12-01"

start_date_dt = datetime.strptime(start_date, "%Y-%m-%d")
end_date_dt   = start_date_dt + timedelta(days=7)
end_date = end_date_dt.strftime("%Y-%m-%d")
list_df_data = list()

while end_date_dt <= datetime.now() + timedelta(days=1):
    print(f"Récupération des données du {start_date} au {end_date}")
    data = get_consumption_load(PRM, start_date, end_date, API_TOKEN)
    start_date_dt = end_date_dt
    start_date = start_date_dt.strftime("%Y-%m-%d")
    end_date_dt   = start_date_dt + timedelta(days=7)
    end_date = end_date_dt.strftime("%Y-%m-%d")
    list_data = data['interval_reading']
    df = pd.DataFrame(list_data)
    list_df_data.append(df)

last_date = datetime.strptime(data['interval_reading'][-1]['date'], "%Y-%m-%d %H:%M:%S")

if last_date < datetime.now():
    start_date = last_date.strftime("%Y-%m-%d")
    end_date_dt   = last_date + timedelta(days=1)
    end_date = end_date_dt.strftime("%Y-%m-%d")
    data = get_consumption_load(PRM, start_date, end_date, API_TOKEN)
    list_data = data['interval_reading']
    df = pd.DataFrame(list_data)
    list_df_data.append(df)


dataconso = pd.concat(list_df_data, ignore_index=True)

# Convertir la colonne 'date' en datetime
dataconso['date'] = pd.to_datetime(dataconso['date'])

# Convertir 'value' en numérique (int ou float)
dataconso['value'] = pd.to_numeric(dataconso['value'])

# Définir l'index temporel
dataconso.set_index('date', inplace=True)

In [27]:
import psycopg2

# Connexion à PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="electricite",
    user="postgres",
    password="Labrax_007"
)


In [29]:
# Créer un curseur
cur = conn.cursor()

In [ ]:
# Créer une table de test
cur.execute("""
CREATE TABLE IF NOT EXISTS courbecharge (
    id SERIAL PRIMARY KEY,
    timestamp TIMESTAMP,
    value FLOAT
)
""")

conn.commit() 

In [40]:
import pandas as pd
from psycopg2 import sql

# Vérifie et convertis l'index si nécessaire
print("Type de l'index avant conversion :", type(dataconso.index))
dataconso.index = pd.to_datetime(dataconso.index)  # Force la conversion en DatetimeIndex
print("Type de l'index après conversion :", type(dataconso.index))

# Insertion des données
for index, row in dataconso.iterrows():
    try:
        cur.execute(
            sql.SQL("INSERT INTO courbecharge (timestamp, value) VALUES (%s, %s)"),
            (index, row['value'])
        )
    except Exception as e:
        print(f"Erreur à l'insertion de la ligne {index}: {e}")
        conn.rollback()
    else:
        conn.commit()


Type de l'index avant conversion : <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
Type de l'index après conversion : <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
Erreur à l'insertion de la ligne 2024-12-01 01:00:00: ERREUR:  la transaction est annulée, les commandes sont ignorées jusqu'à la fin du bloc
de la transaction



In [32]:
cur.execute("SELECT current_schema();")
print(cur.fetchone())

('public',)


In [41]:
from psycopg2 import sql

# Requête pour supprimer les doublons (exemple : garde la première occurrence)
query = sql.SQL("""
    DELETE FROM courbecharge
    WHERE id NOT IN (
        SELECT MIN(id)
        FROM courbecharge
        GROUP BY timestamp
    );
""")

try:
    cur.execute(query)
    conn.commit()
    print("Les doublons ont été supprimés avec succès.")
except Exception as e:
    print(f"Erreur lors de la suppression des doublons : {e}")
    conn.rollback()

Les doublons ont été supprimés avec succès.


In [42]:
conn.close()